In [1]:
import json
import numpy as np
import sys
sys.path.append('../src')
import cpu

# https://github.com/TomHarte/ProcessorTests/tree/main/nes6502

with open('C:/Users/David/Documents/Coding/daveNES/tests/ProcessorTests-main/nes6502/v1/28.json') as f:
    test = json.load(f)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


`! pytest --html="report_%date:/=-%_%time::=-%.html"`

In [2]:
subtest = 0

In [3]:
test[0]

{'name': '28 87 5a',
 'initial': {'pc': 50529,
  's': 58,
  'a': 28,
  'x': 193,
  'y': 77,
  'p': 229,
  'ram': [[50529, 40], [50530, 135], [50531, 90], [314, 173], [315, 136]]},
 'final': {'pc': 50530,
  's': 59,
  'a': 28,
  'x': 193,
  'y': 77,
  'p': 168,
  'ram': [[314, 173], [315, 136], [50529, 40], [50530, 135], [50531, 90]]},
 'cycles': [[50529, 40, 'read'],
  [50530, 135, 'read'],
  [314, 173, 'read'],
  [315, 136, 'read']]}

In [4]:
test[subtest]['initial']['ram']

[[50529, 40], [50530, 135], [50531, 90], [314, 173], [315, 136]]

In [5]:
def init_daveNES(test):
    daveNES = cpu.MOS6502(debug = False)
    daveNES.initialise_RAM()

    # Load test program
    for val in test['initial']['ram']:
        print(f'{val[0]}, {val[1]}')
        daveNES.ram.memory[val[0]] = np.uint8(val[1])
    daveNES.r_program_counter = test['initial']['pc']
    daveNES.r_stack_pointer = test['initial']['s']
    daveNES.r_accumulator = test['initial']['a']
    daveNES.r_index_X = test['initial']['x']
    daveNES.r_index_Y = test['initial']['y']
    daveNES.value_to_status(test['initial']['p'])

    return daveNES

In [6]:
daveNES = init_daveNES(test[subtest])

50529, 40
50530, 135
50531, 90
314, 173
315, 136


In [7]:
daveNES.step_program()
daveNES.r_status['flag_B1'] = 1

0x28, PLP
PC: 0xc561, SP: 0x3a, A: 0x1c, X: 0xc1, Y: 0x4d, [1, 1, 1, 0, 0, 1, 0, 1]


In [8]:
print(f'{test[subtest]["final"]["pc"]} : {daveNES.r_program_counter}')
print(f'{test[subtest]["final"]["s"]} : {daveNES.r_stack_pointer}')
print(f'{test[subtest]["final"]["a"]} : {daveNES.r_accumulator}')
print(f'{test[subtest]["final"]["x"]} : {daveNES.r_index_X}')
print(f'{test[subtest]["final"]["y"]} : {daveNES.r_index_Y}')
print(f'{test[subtest]["final"]["p"]:08b} : {"".join(str(int(daveNES.r_status[k])) for k in daveNES.r_status.keys())[::-1]}')

50530 : 50530
59 : 59
28 : 28
193 : 193
77 : 77
10101000 : 10011000


In [9]:
print(f'{136:08b}')

10001000


In [10]:
for i in range(-100, 100):
    val = daveNES.ram.memory[daveNES.r_stack_pointer + i]
    if val != 0:
        print(val)

In [11]:
keys = []
for k in daveNES.r_status.keys():
    keys += [k]

In [12]:
keys[::-1]

['flag_N',
 'flag_V',
 'flag_B1',
 'flag_B0',
 'flag_D',
 'flag_I',
 'flag_Z',
 'flag_C']

In [13]:
"".join(str(int(daveNES.r_status[k])) for k in daveNES.r_status.keys())[::-1]

'10011000'

In [14]:
np.uint8(64) & 0b1000_0000

0